# OpenWPM - Análisis estilo Lightbeam

Este Notebook:
1. Carga los datos de una ejecución previa de OpenWPM (archivo SQLite).
2. Identifica las peticiones a "terceros" (third-party) al comparar el dominio de la página visitada vs. el dominio del recurso.
3. Contesta a preguntas habituales tipo Lightbeam:
   - Número total de terceros
   - Página con más terceros
   - Dos páginas con más terceros en común
   - Tercero más frecuente


In [ ]:
import sqlite3
import pandas as pd
from urllib.parse import urlparse
from itertools import combinations

DB_PATH = "/workspaces/OpenWPM/datadir/crawl-data.sqlite"

# 1. Conectamos con la base de datos SQLite que generó OpenWPM
conn = sqlite3.connect(DB_PATH)

# 2. Cargamos la tabla http_requests, donde se registran las peticiones
df_requests = pd.read_sql_query("SELECT * FROM http_requests", conn)
conn.close()

print("Columnas disponibles en http_requests:\n", df_requests.columns.tolist())
print("\nEjemplo de filas:")
display(df_requests.head(5))

### 3. Normalizar dominios
Vamos a extraer el dominio de la página visitada (columna `top_level_url`) y el dominio del recurso (`url`) para identificar si es un tercero.

In [ ]:
def get_domain_from_url(u: str) -> str:
    if not u:
        return ""
    parsed = urlparse(u)
    return parsed.netloc.lower()

df_requests["page_domain"] = df_requests["top_level_url"].apply(get_domain_from_url)
df_requests["req_domain"]  = df_requests["url"].apply(get_domain_from_url)

print(df_requests[["top_level_url", "page_domain", "url", "req_domain"]].head())

### 4. Identificar peticiones a terceros
Definimos que una petición es de "terceros" si el dominio del recurso (`req_domain`) difiere del dominio de la página (`page_domain`).

In [ ]:
# Creamos una columna booleana is_thirdparty
df_requests["is_thirdparty"] = df_requests.apply(
    lambda row: row["req_domain"] != row["page_domain"], axis=1
)
df_third = df_requests[df_requests["is_thirdparty"]].copy()
print(f"Total de peticiones: {len(df_requests)}")
print(f"Total de peticiones a terceros: {len(df_third)}")

### 5. Calcular el número total de terceros
Obtenemos los dominios de terceros únicos:

In [ ]:
all_third_parties = df_third["req_domain"].unique()
num_third_parties = len(all_third_parties)

print("Número total de dominios terceros únicos:", num_third_parties)

### 6. Página con más terceros
Agrupamos por la página visitada y contamos cuántos terceros *distintos* aparecen en cada una.

In [ ]:
# Agrupamos en sets para no contar repetidos
grouped_sets = df_third.groupby("page_domain")["req_domain"].agg(set)
# El resultado es un Series: page_domain -> set({varios_terceros})

df_counts = grouped_sets.apply(len).reset_index(name="count_third")
page_with_max = df_counts.loc[df_counts["count_third"].idxmax()]
print("Página con más terceros:", page_with_max["page_domain"], 
      "\nNúmero de terceros:", page_with_max["count_third"])

### 7. Dos páginas con más terceros en común
Para cada par de sitios, calculamos la intersección de sus sets de dominios terceros:

In [ ]:
page_to_thirdset = grouped_sets.to_dict()  # dict: { 'page_domain' -> set({terceros...}) }

all_pages = list(page_to_thirdset.keys())
max_shared = -1
best_pair = (None, None)

for p1, p2 in combinations(all_pages, 2):
    shared = page_to_thirdset[p1].intersection(page_to_thirdset[p2])
    scount = len(shared)
    if scount > max_shared:
        max_shared = scount
        best_pair = (p1, p2)

print("Las dos páginas con más terceros en común son:", best_pair)
print("Número de terceros compartidos:", max_shared)

### 8. Tercero más frecuente
Buscamos en cuántos **sitios** aparece cada dominio tercero.

In [ ]:
# Transformamos page -> set(terceros) a una lista [(page, dom), ...]
pairs = []
for page, sdoms in page_to_thirdset.items():
    for d in sdoms:
        pairs.append((page, d))

df_pairs = pd.DataFrame(pairs, columns=["page_domain", "third_domain"])
# Contamos en cuántas páginas aparece cada third_domain
counts = df_pairs["third_domain"].value_counts()

most_common_third = counts.index[0]
freq = counts.iloc[0]
print("Tercero más frecuente:", most_common_third)
print("Aparece en", freq, "páginas distintas de", len(all_pages), "totales.")

## 9. Exportar resultados
En esta celda generaremos un archivo JSON con la información clave (número total, página con más terceros, par de páginas con mayor intersección y la lista de esos terceros, etc.).

In [ ]:
import json

# A) Número total de terceros
num_total = num_third_parties

# B) Página con más terceros
page_max_name = page_with_max["page_domain"]
page_max_count = page_with_max["count_third"]

# C) Dos páginas con más terceros en común
p1, p2 = best_pair
shared_thirds_list = list(page_to_thirdset[p1].intersection(page_to_thirdset[p2]))

# D) Tercero más frecuente
most_freq_third = most_common_third
most_freq_third_count = freq

analysis_dict = {
    "num_third_parties": int(num_total.item()),
    "page_with_max_thirds": page_max_name,
    "count_on_that_page": int(page_max_count.item()),
    "pages_with_most_shared_thirds": {
        "page1": p1,
        "page2": p2,
        "shared_count": int(max_shared.item()),
        "shared_third_parties": list(shared_thirds_list)
    },
    "most_frequent_third": most_freq_third,
    "freq_third": int(most_freq_third_count.item())
}

with open("analysis_results.json", "w", encoding="utf-8") as f:
    json.dump(analysis_dict, f, indent=2, ensure_ascii=False)

print("analysis_results.json creado con la información principal.")

## 10. Exportar la lista de terceros por página
Generamos un CSV que muestre, para cada página, qué terceros se encontraron.

In [ ]:
# Partimos de df_third, que contiene solo peticiones a terceros
# Agrupamos en sets y convertimos a lista
df_thirds_by_page = (
    df_third.groupby("page_domain")["req_domain"]
    .agg(lambda s: list(set(s)))
    .reset_index()
    .rename(columns={"req_domain": "third_parties_list"})
)

df_thirds_by_page.to_csv("thirds_by_page.csv", index=False)
print("Archivo thirds_by_page.csv creado correctamente.")

## Conclusiones
Con este Notebook hemos:
- Cargado los datos generados por OpenWPM.
- Identificado peticiones de terceros comparando dominios.
- Obtenido:
  1. Número total de terceros
  2. Página con más terceros
  3. Dos páginas con más terceros en común
  4. Tercero que aparece en más páginas

Puedes ajustar la lógica de "tercero" (e.g., ignorar subdominios), añadir más análisis, o usar cualquier otra tabla (`http_responses`, `cookies`, etc.) según necesites.